In [ ]:
import numpy as np
import torch
from tqdm import tqdm

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import seaborn
import sys
seaborn.set()

import warnings
warnings.filterwarnings("ignore")

import reservoir
import reckernel
import kuramoto
%load_ext autoreload
%autoreload 2
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Data generation

In [ ]:
L = 22 / (2 * np.pi)  # length
N = 100  # space discretization step
dt = 0.25  # time discretization step
N_train = 10000
N_test = 10000
N_init = 1000  # remove the initial points
tend = (N_train + N_test + N_init) * dt

dns = kuramoto.KS(L=L, N=N, dt=dt, tend=tend)
dns.simulate()
u = dns.uu[ninittransients:] / np.sqrt(N)
[u_train, u_test, _] = np.split(u, [N_data_train, N_data_train+N_data_test], axis=0)

In [ ]:
N_plot = 1000
u_plot = u[:N_plot,:]

plt.figure()
plt.imshow(u.T)
plt.colorbar()
plt.grid(b=None)
plt.xlabel(r"$n, \quad t={:}n$".format(dt))
plt.ylabel(r"$x$")
plt.title("Kuramoto-Sivashisky time series")

# Stability for Reservoir Computing

In [ ]:
input_data = torch.from_numpy(u_train).to(device)
input_len = 100
input_data = input_data[:input_len, :]

n_input = 1 # number of time series
input_len, input_dim = input_data.shape # time steps t
input_scale = 0.01  # variance of input
res_scale = 0.8 # variance of reservoir

n_res = 1000
torch.manual_seed(1)
initial_state1 = torch.randn(n_res).to(device) / np.sqrt(n_res)
model = reservoir.ESN(input_dim, res_size=n_res, res_scale=res_scale, input_scale=input_scale, f='erf')
X1 = model.forward(input_data, initial_state=initial_state1).to(device)

torch.manual_seed(2)
initial_state2 = torch.randn(n_res).to(device) / np.sqrt(n_res)
model = reservoir.ESN(input_dim, res_size=n_res, res_scale=res_scale, input_scale=input_scale, f='erf')
X2 = model.forward(input_data, initial_state=initial_state2).to(device)

In [ ]:
diff = X2 - X1
plt.plot(torch.mean(torch.abs(diff)**2, axis=1).cpu());

# Stability for Recurrent Kernels

In [ ]:
input_data = torch.from_numpy(u_train).to(device)
input_len = 150
input_data = input_data[:input_len, :]

n_input = 1 # number of time series
input_len, input_dim = input_data.shape # time steps t
input_scale = 0.01  # variance of input
res_scale = 0.8 # variance of reservoir

model = reckernel.RK(input_dim, res_scale=res_scale, input_scale=input_scale, f='arcsin', n_iter=50)
dist2 = model.stability_test(input_data)

In [ ]:
plt.plot(dist2.cpu());